## Import Libraries

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import random
import math
import time

import operator
from queue import PriorityQueue

## Google Drive

In [ ]:
import os
!pip install wandb
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('./drive/MyDrive/Colab Notebooks/CS6910/Assignment 3/')

     |████████████████████████████████| 1.8MB 8.0MB/s 
     |████████████████████████████████| 133kB 54.8MB/s 
     |████████████████████████████████| 163kB 49.9MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=9cc485683fbfe62a912905dfaaee75cb32555229a16db1962c6141ef8ab021a3
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=b8c406f51fc4a57e63dceaddcf8e78ba559c0ec80ee205d43849a2e9ec245f7f
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
Mounted at /content/drive


## Wandb

In [ ]:
!pip install wandb --upgrade
import wandb
!wandb login

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.30)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Set up device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

## Assign model name

In [ ]:
modelName = 'Encoder_Decoder_attention_Dakshina_Hi'

## Set up Dataset

In [ ]:
with open(r"./drive/MyDrive/Colab Notebooks/CS6910/Assignment 3/hi/lexicons/hi.translit.sampled.train.tsv", 'r', encoding="utf8") as f:
    train_lines = f.read().split("\n")
    
with open(r"./drive/MyDrive/Colab Notebooks/CS6910/Assignment 3/hi/lexicons/hi.translit.sampled.dev.tsv", 'r', encoding="utf8") as f:
    val_lines = f.read().split("\n")

In [ ]:
train_num_samples = len(train_lines)
val_num_samples = len(val_lines)

In [ ]:
train_input_texts = []
train_target_texts = []

val_input_texts = []
val_target_texts = []

input_characters = set()
target_characters = set()

In [ ]:
## Train Data ##

for line in train_lines[: min(train_num_samples, len(train_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    train_input_texts.append(input_text)
    train_target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters.add(' ')
target_characters.add(' ')
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

train_max_encoder_seq_length = max([len(txt) for txt in train_input_texts])
train_max_decoder_seq_length = max([len(txt) for txt in train_target_texts])

print("Number of train samples:", len(train_input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)

print("Max train sequence length for inputs:", train_max_encoder_seq_length)
print("Max train sequence length for outputs:", train_max_decoder_seq_length)

Number of train samples: 44204
Number of unique input tokens: 27
Number of unique output tokens: 66
Max train sequence length for inputs: 20
Max train sequence length for outputs: 21


In [ ]:
## Val Data ##

for line in val_lines[: min(val_num_samples, len(val_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)
    
val_max_encoder_seq_length = max([len(txt) for txt in val_input_texts])
val_max_decoder_seq_length = max([len(txt) for txt in val_target_texts])

print("Number of val samples:", len(val_input_texts))

print("Max val sequence length for inputs:", val_max_encoder_seq_length)
print("Max val sequence length for outputs:", val_max_decoder_seq_length)

Number of val samples: 4358
Max val sequence length for inputs: 18
Max val sequence length for outputs: 16


In [ ]:
# token indices
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data_train = np.zeros(
    (len(train_input_texts), train_max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data_train = np.zeros(
    (len(train_input_texts), train_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)


encoder_input_data_val = np.zeros(
    (len(val_input_texts), train_max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data_val = np.zeros(
    (len(val_input_texts), train_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [ ]:
## Train Data ##

for i, (input_text, target_text) in enumerate(zip(train_input_texts, train_target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data_train[i, t, input_token_index[char]] = 1. 
    encoder_input_data_train[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        decoder_input_data_train[i, t, target_token_index[char]] = 1.
    decoder_input_data_train[i, t + 1:, target_token_index[' ']] = 1.
    
## Val Data ##

for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data_val[i, t, input_token_index[char]] = 1. 
    encoder_input_data_val[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        decoder_input_data_val[i, t, target_token_index[char]] = 1.
    decoder_input_data_val[i, t + 1:, target_token_index[' ']] = 1.

In [ ]:
# Convert data to tensors so that u can pass through dataloaders

encoder_inp_train = torch.stack([torch.from_numpy(np.array(i)) for i in encoder_input_data_train])
decoder_inp_train = torch.stack([torch.from_numpy(np.array(i)) for i in decoder_input_data_train])

encoder_inp_val = torch.stack([torch.from_numpy(np.array(i)) for i in encoder_input_data_val])
decoder_inp_val = torch.stack([torch.from_numpy(np.array(i)) for i in decoder_input_data_val])

In [ ]:
batch_size = 64

train_dataset = torch.utils.data.TensorDataset(encoder_inp_train, decoder_inp_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

val_dataset = torch.utils.data.TensorDataset(encoder_inp_val, decoder_inp_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)#, shuffle = True)

## Build Model

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        timestep = encoder_outputs.size(0)
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        attn_energies = self.score(h, encoder_outputs) 
        return F.softmax(attn_energies, dim=1).unsqueeze(1) 

    def score(self, hidden, encoder_outputs):
        energy = F.relu(self.attn(torch.cat([hidden, encoder_outputs], 2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        energy = torch.bmm(v, energy)
        return energy.squeeze(1)

In [ ]:
class Encoder(nn.Module):
    def __init__(self,  num_encoder_tokens, hid_dim, n_layers, dropout, enc_embedding_dim = 0, module='LSTM'):
        super(Encoder,self).__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.module = module
        self.embedding_dim = enc_embedding_dim
        self.enc_inp = num_encoder_tokens
        if self.embedding_dim !=0:
            self.enc_inp = self.embedding_dim
            self.embedding = nn.Embedding(num_encoder_tokens, self.embedding_dim)
            
        if self.module =='LSTM':
            self.rnn = nn.LSTM(self.enc_inp, hid_dim, n_layers, dropout = dropout)
        elif self.module == 'RNN':
            self.rnn = nn.RNN(self.enc_inp, hid_dim, n_layers, dropout = dropout)
        elif self.module == 'GRU':
            self.rnn = nn.GRU(self.enc_inp, hid_dim, n_layers, dropout = dropout)
        
    def forward(self, inp):
        
        inp = inp.transpose(0,1)  # check input dimensions before embedding. Currently : (T, N)
        if self.embedding_dim !=0:
            inp = inp.argmax(2)
            inp = self.embedding(inp)
            
        outputs, hidden_cell = self.rnn(inp)
        
        return outputs, hidden_cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, num_decoder_tokens, hid_dim, n_layers, dropout, dec_embedding_dim = 0, module='LSTM', atten=False):
        super(Decoder,self).__init__()
        
        self.output_dim = num_decoder_tokens
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.module = module
        self.atten = atten
        self.embedding_dim = dec_embedding_dim
        self.dec_inp = num_decoder_tokens
        
        if self.embedding_dim !=0:
            self.dec_inp = self.embedding_dim
            self.embedding = nn.Embedding(num_decoder_tokens, self.embedding_dim)
        
        if self.atten == False:
            if module=='LSTM':
                self.rnn = nn.LSTM(self.dec_inp, hid_dim, n_layers, dropout = dropout)
            if module=='RNN':
                self.rnn = nn.RNN(self.dec_inp, hid_dim, n_layers, dropout = dropout)
            if module=='GRU':
                self.rnn = nn.GRU(self.dec_inp, hid_dim, n_layers, dropout = dropout)
                
            self.fc_out = nn.Linear(self.hid_dim, self.output_dim)
                
        else:
            self.attention = Attention(self.hid_dim)
            
            if module=='LSTM':
                self.rnn = nn.LSTM(self.hid_dim + self.dec_inp, hid_dim, n_layers, dropout = dropout)
            if module=='RNN':
                self.rnn = nn.RNN(self.hid_dim + self.dec_inp, hid_dim, n_layers, dropout = dropout)
            if module=='GRU':
                self.rnn = nn.GRU(self.hid_dim + self.dec_inp, hid_dim, n_layers, dropout = dropout)
                
            self.fc_out = nn.Linear(self.hid_dim * 2, self.output_dim)
        
        
        
    def forward(self, inp, hidden_cell, encoder_states):
        
        if isinstance(hidden_cell, tuple):
            hidden = hidden_cell[0]
            cell = hidden_cell[1]
        else:
            hidden = hidden_cell
        
        if self.embedding_dim !=0:
            inp = inp.argmax(2)
            inp = self.embedding(inp)
        
        if self.atten == False:
            if self.module == 'LSTM':
                output, hidden = self.rnn(inp, (hidden, cell))
            else:
                output, hidden = self.rnn(inp,hidden)

            prediction = self.fc_out(output.squeeze(0))
            return prediction, hidden
        
        else:
            attn_weights = self.attention(hidden[-1], encoder_states)  # why -1 ? wat is dim of hidden?
            context = attn_weights.bmm(encoder_states.transpose(0, 1))
            context = context.transpose(0, 1)
            rnn_inp = torch.cat([inp, context], 2) 
            
            if self.module == 'LSTM':
                output, hidden = self.rnn(rnn_inp, (hidden, cell))
            else:
                output, hidden = self.rnn(rnn_inp, hidden)
            
            output = output.squeeze(0) 
            context = context.squeeze(0)
            output = self.fc_out(torch.cat([output, context], 1))
            
            return output, hidden, attn_weights

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.atten = self.decoder.atten
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, to_train, teacher_forcing_ratio = 0.5, beam_width = 3):
        
        if to_train == True:
        
            trg = trg.transpose(0,1)
            batch_size = trg.shape[1]
            trg_len = trg.shape[0]
            trg_vocab_size = self.decoder.output_dim
            
            outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

            enc_output, hidden_cell = self.encoder(src)
            inp = trg[0,:]

            for t in range(1, trg_len):
                
                if self.atten == False:
                    prediction, hidden_cell = self.decoder(inp.unsqueeze(0), hidden_cell, enc_output) # recursively set hidden, cell
                else:
                    prediction, hidden_cell, atten_weights = self.decoder(inp.unsqueeze(0), hidden_cell, enc_output)
                outputs[t] = prediction
                teacher_force = random.random() < teacher_forcing_ratio
                top1 = prediction.argmax(1)
                top1_one_hot = torch.zeros_like(prediction).to(self.device)
                top1_one_hot[:,top1] = 1.

                inp = trg[t] if teacher_force else top1_one_hot

            return outputs
        
        else :
            batch_size = trg.shape[0]
            trg_len = trg.shape[1]
            trg_vocab_size = self.decoder.output_dim

            enc_output, hidden_cell = self.encoder(src)
            outputs = self.beam_decode(beam_width, trg, hidden_cell, enc_output)
            
            return outputs
        
    def beam_decode(self, beam_width, target_tensor, decoder_hiddens, encoder_outputs=None):
        
        target_tensor = target_tensor.transpose(0,1)
        beam_width = beam_width
        topk = 1
        decoded_batch = []
        
        batch_loss = []
        EOS_token = target_token_index['\n']
        
        criterion_infer = torch.nn.CrossEntropyLoss(ignore_index = target_token_index[' '])
        
        for idx in range(target_tensor.size(1)):
            if isinstance(decoder_hiddens, tuple):
                decoder_hidden = (
                    decoder_hiddens[0][:, idx, :].contiguous().unsqueeze(1), decoder_hiddens[1][:, idx, :].contiguous().unsqueeze(1))
            else:
                decoder_hidden = decoder_hiddens[:, idx, :].contiguous().unsqueeze(1)
                
            encoder_output = encoder_outputs[:, idx, :].unsqueeze(1)
            decoder_input_token =  target_token_index["\t"]
            endnodes = []
            number_required = min((topk + 1), topk - len(endnodes))

            node = BeamSearchNode(decoder_hidden,   None,       decoder_input_token, 0,     1   , 0)
            nodes = PriorityQueue()

            nodes.put((-node.eval(), node))
            qsize = 1

            while True:
                if qsize > train_max_decoder_seq_length * beam_width: 
                    break

                score, n = nodes.get()
                decoder_input_token = n.wordid
                #print("wordid: ",n.wordid)
                decoder_hidden = n.h
                
                decoder_input = torch.zeros((1, 1, num_decoder_tokens)).to(self.device)
                decoder_input[0, 0, decoder_input_token] = 1.
                if n.wordid == EOS_token and n.prevNode != None:
                    
                    endnodes.append((score, n))
                    if len(endnodes) >= number_required:
                        break
                    else:
                        continue

                if self.atten == False:
                    decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_output)
                
                else:
                    decoder_output, decoder_hidden, atten_weights = self.decoder(decoder_input, decoder_hidden, encoder_output)
                
                log_softmax =  F.log_softmax(decoder_output, dim=1)
                log_prob, indexes = torch.topk(log_softmax, beam_width)
                #log_prob =  F.log_softmax(log_prob, dim=1)
                nextnodes = []
                # Below seems okay :-)
                #print("Verify target_tensor shape to check argmax(0)",target_tensor[n.leng, idx].shape)
                loss_at_t = criterion_infer(decoder_output, target_tensor[n.leng, idx].argmax(0).unsqueeze(0))
                
                for new_k in range(beam_width):
                    decoded_t = indexes[0][new_k].view(-1)
                    log_p = log_prob[0][new_k].item()
                    #print("decoded_t: ",decoded_t)
                    #print("decoded_t: ",n.wordid)
                    node = BeamSearchNode(decoder_hidden, n, decoded_t.item(), n.logp + log_p, n.leng + 1, n.loss + loss_at_t)
                    score = -node.eval()
                    
                    if n.leng < train_max_decoder_seq_length - 1:
                        nextnodes.append((score, node))
                    
                    else:
                        endnodes.append((score,node))

                for i in range(len(nextnodes)):
                    score, nn = nextnodes[i]
                    nodes.put((score, nn))
                    
                qsize += len(nextnodes) - 1
            
            if len(endnodes) == 0:
                endnodes = [nodes.get() for _ in range(topk)]

            utterances = []
            utterances_loss = []
            for score, n in sorted(endnodes, key=operator.itemgetter(0)):
                utterance = []
                utterance.append(n.wordid)
                loss_sum = n.loss
                utterances_loss.append(loss_sum)
                while n.prevNode != None:
                    n = n.prevNode
                    utterance.append(n.wordid)
                
                utterance = utterance[::-1]
                utterances.append(utterance)
                #print(utterance)
            decoded_batch.append(utterances)
            batch_loss.append(utterances_loss)
            
        return decoded_batch, batch_loss


class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length, loss):
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length
        self.loss = loss

    def eval(self, alpha=1.0):
        reward = 0
        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward 

    def __lt__(self, other):
        return self.leng < other.leng

    def __gt__(self, other):
        return self.leng > other.leng

## Helpers to decode back sentance

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

## Functions to match output sequence to calculate accuracy

In [ ]:
def no_correct_in_batch_train(target, output):
    target = target.transpose(0,1)
    truth = output.argmax(2).transpose(0,1)
    no_correct = 0
    batch_size = target.shape[0]
    trgt_length = target.shape[1]
    for seq in range(batch_size):
        #decoded_sen = ""
        for char in range(1, trgt_length):
            if target[seq,char] == target_token_index['\n']:
                no_correct += 1
                break
            if target[seq,char] != truth[seq,char]:
                break
            #decoded_sen = decoded_sen + reverse_target_char_index[target[seq,char]]
        #print(decoded_sen)
    return no_correct, batch_size

In [ ]:
def no_correct_in_batch_infer(target, output):
    target = target.transpose(0,1)
    no_correct = 0
    batch_size = target.shape[0]
    trgt_length = target.shape[1]
    
    # Verify batch_size  ------ Seems Okay
    #print("In Infer, BS = 256 , TL = 21: ",target.shape)
    #print("Target BS: ", batch_size)
    #print("Output BS: ", len(output))
    for seq in range(batch_size):
        #true_sen = ""
        #decoded_sen = ""
        
        for char in range(1,trgt_length):
            
            if target[seq,char] == target_token_index['\n']:
                no_correct += 1
                break
            if char== len(output[seq][0]) or target[seq,char] != output[seq][0][char]:
                break
            #true_sen = true_sen + reverse_target_char_index[target[seq,char].item()]
            #decoded_sen = decoded_sen + reverse_target_char_index[output[seq][0][char]]
        
        #print("true output is ",true_sen)
        #print("decoded output is ",decoded_sen)
    return no_correct, batch_size

## Train Function

In [ ]:
def train(model, iterator, optimizer, criterion, clip, teacher_forcing_ratio):
    
    model.train()
    
    epoch_loss = 0
    
    total_no_correct = 0
    total_samples = 0
    for i, (src,trg) in enumerate(iterator):
        
        optimizer.zero_grad()
        src, trg = src.to(device), trg.to(device)
        
        
        output = model(src, trg, teacher_forcing_ratio = teacher_forcing_ratio, to_train = True).to(device) 
        trg = trg.transpose(0,1)
        
        trg = trg.argmax(2)
        no_correct , samples = no_correct_in_batch_train(trg, output)
        total_no_correct += no_correct
        total_samples += samples
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].reshape(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator), total_no_correct/total_samples

## Evaluate function

In [ ]:
def evaluate(model, iterator, beam_width):
    
    model.eval()
    
    epoch_loss = 0
    total_no_correct = 0
    total_samples = 0
    with torch.no_grad():
    
        for i, (src,trg) in enumerate(iterator):

            src, trg = src.to(device), trg.to(device)

            output = model( src, trg, beam_width = beam_width, to_train= False)
            
            trg = trg.transpose(0,1)

            trg = trg.argmax(2)
            no_correct , samples = no_correct_in_batch_infer(trg, output[0])
            total_no_correct += no_correct
            total_samples += samples
            
            loss = 0
            total_chars = 0
            for sample in range(len(output[1])):
                for utterances in range(len(output[1][sample])):
                    loss+= output[1][sample][utterances] 
                    total_chars += len(output[0][sample][utterances])
            
            epoch_loss += loss / total_chars
            
    return epoch_loss / len(iterator), total_no_correct/total_samples

## Function to calculate time

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Start Training and evaluating the model

In [ ]:
def train_evaluate(model, train_loader,val_loader, optimizer, criterion, CLIP, N_EPOCHS, teacher_forcing_ratio, beam_width):
    teacher_forcing_ratio = teacher_forcing_ratio
    
    for epoch in range(N_EPOCHS):

        start_time = time.time()
        
        train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, CLIP, teacher_forcing_ratio)
        valid_loss, valid_accuracy = evaluate(model, val_loader, beam_width)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\tTrain Accuracy: {train_accuracy:.3f}')
        print(f'\tVal. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
        print(f'\tVal Accuracy: {valid_accuracy:.3f}')
        
        wandb.log({'epoch': epoch,'train loss': train_loss,'train PPL':math.exp(train_loss), 'train accuracy': train_accuracy,
                   'val loss': valid_loss,'valid PPL':math.exp(valid_loss), 'val accuracy': valid_accuracy})
        
    return model

In [ ]:
def sp_train():
    config_defaults = {
        'epochs': 20,
        'cell_type':'LSTM',
        #'enc_embedding_dim': 10,
        #'dec_embedding_dim': 30,
        'no_encoder_decoder_layers': 1 ,
        'hidden_layer_size': 256,
        'dropout':0.5,
        'beam_search_size': 3,
        #'lr': 1e-3,
        #'optimizer': 'adam',
        'teacher_forcing_ratio': 0.5
    }
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    run_name="cell:["+config.cell_type+"] layers:["+str(config.no_encoder_decoder_layers)+"] HL_size:["+str(config.hidden_layer_size)+"] dp:["+str(config.dropout)+"] beam_width:["+str(config.beam_search_size)+"] tfr:["+str(config.teacher_forcing_ratio)+"]"
    wandb.run.name=run_name
    teacher_forcing_ratio = config.teacher_forcing_ratio
    CLIP = 1
    enc_embedding_dim = 0
    dec_embedding_dim = 0
    enc = Encoder( num_encoder_tokens, config.hidden_layer_size, n_layers = config.no_encoder_decoder_layers, enc_embedding_dim = enc_embedding_dim, dropout = config.dropout, module=config.cell_type)
    dec = Decoder( num_decoder_tokens, config.hidden_layer_size, n_layers = config.no_encoder_decoder_layers, dec_embedding_dim = dec_embedding_dim, dropout = config.dropout, module=config.cell_type, atten = True)

    model = Seq2Seq(enc, dec, device).to(device)
    learning_rate = 1e-3
    #if config.optimizer == 'adam':
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    
    #elif config.optimizer == 'RMSprop':
    #  optimizer = optim.RMSprop(model.parameters(), lr = config.learning_rate)
    #scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
    criterion = nn.CrossEntropyLoss(ignore_index = target_token_index[' '])
    
    train_evaluate(model, train_loader, val_loader, optimizer, criterion, CLIP, config.epochs, teacher_forcing_ratio, config.beam_search_size)

In [ ]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'val accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values':[20]
        },
        'cell_type':{
            'values': [
                       'GRU', 
                       'LSTM'
                       ]
        },
        'no_encoder_decoder_layers': {
            'values':[1,2,3] 
        },
        'hidden_layer_size':{
            'values':[64,256,512]
        },
        'dropout':{
            'values': [0.20,0.50]
        },
        'beam_search_size': {
            'values':[3,4]
        },
        'teacher_forcing_ratio': {
            'values':[0.2,0.5]
        }
    }
}

In [ ]:
#wandb.agent(sweep_id = 'g9ym6ooa', project=modelName+"_Sweep_1", function = sp_train)

wandb: Agent Starting Run: 21j0x79y with config:
wandb: 	beam_search_size: 2
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.6573622668013036
wandb: 	epochs: 15
wandb: 	hidden_layer_size: 512
wandb: 	no_encoder_decoder_layers: 2
wandb: 	teacher_forcing_ratio: 0.27609870077288134


Epoch: 01 | Time: 1m 21s
	Train Loss: 2.215 | Train PPL:   9.162
	Train Accuracy: 0.024
	Val. Loss: 1.146 |  Val. PPL:   3.147
	Val Accuracy: 0.093
Epoch: 02 | Time: 1m 25s
	Train Loss: 1.130 | Train PPL:   3.094
	Train Accuracy: 0.141
	Val. Loss: 0.876 |  Val. PPL:   2.400
	Val Accuracy: 0.187
Epoch: 03 | Time: 1m 18s
	Train Loss: 0.911 | Train PPL:   2.487
	Train Accuracy: 0.223
	Val. Loss: 0.799 |  Val. PPL:   2.223
	Val Accuracy: 0.281
Epoch: 04 | Time: 1m 19s
	Train Loss: 0.779 | Train PPL:   2.178
	Train Accuracy: 0.284
	Val. Loss: 0.829 |  Val. PPL:   2.290
	Val Accuracy: 0.309
Epoch: 05 | Time: 1m 18s
	Train Loss: 0.681 | Train PPL:   1.975
	Train Accuracy: 0.330
	Val. Loss: 0.860 |  Val. PPL:   2.362
	Val Accuracy: 0.341
Epoch: 06 | Time: 1m 18s
	Train Loss: 0.606 | Train PPL:   1.833
	Train Accuracy: 0.373
	Val. Loss: 0.888 |  Val. PPL:   2.430
	Val Accuracy: 0.372
Epoch: 07 | Time: 1m 17s
	Train Loss: 0.554 | Train PPL:   1.740
	Train Accuracy: 0.405
	Val. Loss: 0.913 |  Val

epoch,14
train loss,0.31189
train PPL,1.366
train accuracy,0.59035
val loss,1.09596
valid PPL,2.99206
val accuracy,0.39927
_runtime,1196
_timestamp,1620844752
_step,14


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train loss,█▄▃▃▂▂▂▂▂▁▁▁▁▁▁
train PPL,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
train accuracy,▁▂▃▄▅▅▆▆▆▇▇▇███
val loss,█▃▁▂▂▃▃▄▄▅▅▅▆▆▇
valid PPL,█▂▁▂▂▃▃▃▃▅▄▅▅▆▇
val accuracy,▁▃▅▆▆▇▇▇▇▇▇▇███
_runtime,▁▂▂▃▃▄▄▅▅▅▆▆▇▇█
_timestamp,▁▂▂▃▃▄▄▅▅▅▆▆▇▇█
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█


wandb: Agent Starting Run: xye116q4 with config:
wandb: 	beam_search_size: 4
wandb: 	cell_type: GRU
wandb: 	dropout: 0.6303525822456986
wandb: 	epochs: 14
wandb: 	hidden_layer_size: 256
wandb: 	no_encoder_decoder_layers: 1
wandb: 	teacher_forcing_ratio: 0.44932953528102826


...

Epoch: 01 | Time: 1m 26s
	Train Loss: 2.029 | Train PPL:   7.607
	Train Accuracy: 0.039
	Val. Loss: 1.036 |  Val. PPL:   2.819
	Val Accuracy: 0.111
Epoch: 02 | Time: 1m 19s
	Train Loss: 0.992 | Train PPL:   2.697
	Train Accuracy: 0.177
	Val. Loss: 1.078 |  Val. PPL:   2.937
	Val Accuracy: 0.277
Epoch: 03 | Time: 1m 17s
	Train Loss: 0.737 | Train PPL:   2.089
	Train Accuracy: 0.263
	Val. Loss: 1.135 |  Val. PPL:   3.111
	Val Accuracy: 0.320
Epoch: 04 | Time: 1m 18s
	Train Loss: 0.624 | Train PPL:   1.866
	Train Accuracy: 0.310
	Val. Loss: 1.200 |  Val. PPL:   3.322
	Val Accuracy: 0.338
Epoch: 05 | Time: 1m 17s
	Train Loss: 0.558 | Train PPL:   1.746
	Train Accuracy: 0.347
	Val. Loss: 1.163 |  Val. PPL:   3.201
	Val Accuracy: 0.374
Epoch: 06 | Time: 1m 17s
	Train Loss: 0.508 | Train PPL:   1.663
	Train Accuracy: 0.374
	Val. Loss: 1.170 |  Val. PPL:   3.222
	Val Accuracy: 0.375
Epoch: 07 | Time: 1m 13s
	Train Loss: 0.468 | Train PPL:   1.596
	Train Accuracy: 0.402
	Val. Loss: 1.187 |  Val

epoch,13
train loss,0.33415
train PPL,1.39675
train accuracy,0.51432
val loss,1.25068
valid PPL,3.49272
val accuracy,0.41831
_runtime,1092
_timestamp,1620855312
_step,13


epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
train loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁
train PPL,█▂▂▂▁▁▁▁▁▁▁▁▁▁
train accuracy,▁▃▄▅▆▆▆▇▇▇▇███
val loss,▁▂▄▆▅▅▆▅▇█▇▇▇█
valid PPL,▁▂▄▆▅▅▆▅▇█▇▇▇█
val accuracy,▁▅▆▆▇▇▇█▇█████
_runtime,▁▂▂▃▃▄▄▅▅▆▆▇▇█
_timestamp,▁▂▂▃▃▄▄▅▅▆▆▇▇█
_step,▁▂▂▃▃▄▄▅▅▆▆▇▇█


wandb: Agent Starting Run: 3vscmtfx with config:
wandb: 	beam_search_size: 4
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.27909888445142167
wandb: 	epochs: 15
wandb: 	hidden_layer_size: 256
wandb: 	no_encoder_decoder_layers: 3
wandb: 	teacher_forcing_ratio: 0.2690853302397402


Epoch: 01 | Time: 1m 38s
	Train Loss: 2.782 | Train PPL:  16.157
	Train Accuracy: 0.001
	Val. Loss: 1.874 |  Val. PPL:   6.515
	Val Accuracy: 0.002
Epoch: 02 | Time: 1m 27s
	Train Loss: 1.714 | Train PPL:   5.554
	Train Accuracy: 0.036
	Val. Loss: 1.176 |  Val. PPL:   3.241
	Val Accuracy: 0.066
Epoch: 03 | Time: 1m 28s
	Train Loss: 1.196 | Train PPL:   3.308
	Train Accuracy: 0.124
	Val. Loss: 1.013 |  Val. PPL:   2.753
	Val Accuracy: 0.188
Epoch: 04 | Time: 1m 23s
	Train Loss: 0.956 | Train PPL:   2.601
	Train Accuracy: 0.199
	Val. Loss: 1.018 |  Val. PPL:   2.767
	Val Accuracy: 0.279
Epoch: 05 | Time: 1m 20s
	Train Loss: 0.834 | Train PPL:   2.302
	Train Accuracy: 0.250
	Val. Loss: 0.994 |  Val. PPL:   2.702
	Val Accuracy: 0.307
Epoch: 06 | Time: 1m 19s
	Train Loss: 0.744 | Train PPL:   2.105
	Train Accuracy: 0.291
	Val. Loss: 0.999 |  Val. PPL:   2.715
	Val Accuracy: 0.335
Epoch: 07 | Time: 1m 20s
	Train Loss: 0.690 | Train PPL:   1.994
	Train Accuracy: 0.320
	Val. Loss: 0.946 |  Val

In [ ]:
sweep_id = wandb.sweep(sweep_config, project=modelName+"_Sweep_1")

Create sweep with ID: ms9uhn1q
Sweep URL: https://wandb.ai/rayanz/Encoder_Decoder_attention_Dakshina_Hi_Sweep_1/sweeps/ms9uhn1q


In [ ]:
wandb.agent(sweep_id, sp_train)

wandb: Agent Starting Run: va572bml with config:
wandb: 	beam_search_size: 4
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.5
wandb: 	epochs: 20
wandb: 	hidden_layer_size: 512
wandb: 	no_encoder_decoder_layers: 2
wandb: 	teacher_forcing_ratio: 0.2


Epoch: 01 | Time: 1m 33s
	Train Loss: 2.128 | Train PPL:   8.400
	Train Accuracy: 0.038
	Val. Loss: 1.072 |  Val. PPL:   2.921
	Val Accuracy: 0.100


wandb: Ctrl + C detected. Stopping sweep.
